# Assignment 3 - Fine Tuning YOLOv5 for Object Detection


## Overview and Prerequisites

This notebooks contains the conducted object detection experiments.

All used datasets are from https://roboflow.com. In order to download dataset from there, you normally need to register. For convenience, we provide all datasets via a Google Drive. Download them and place the zip files in the `src/yolov5/datasets` directory.

* Oxford Pets (by species): https://drive.google.com/uc?id=1bfyssR7CfzTqJ-JAgaZNuM0lgTumDcZy&export=download
* TODO
* TODO

If haven't installed PyTorch yet, remove the comment in the following cell and execute it. In case you need a `CUDA` version different from 11.3, please refer to https://pytorch.org/get-started/locally/.

In [ ]:
#!pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

Install additionally dependencies.

In [ ]:
!pip3 install wandb

In [ ]:
import os

import torch

## Constants and Utiltity Functions

In [ ]:
datasets_base_path = "datasets" # directory where datasets are located
device = 0 if torch.cuda.is_available() else "cpu" # use GPU if available, else CPU

# directories where results are stored (logging, trained model)
project_base_path = "runs" 
project_train = os.path.join(project_base_path, "train")
project_test = os.path.join(project_base_path, "test")

In [ ]:
def adapt_data_yaml_train_val_keys(path):
    """
    Adapts the paths of the train and validation set (keys 'train' and 'val') of a data.yaml file (YOLOv5 format)
    to point to 'path'. The data.yaml file itself also has to be contained in the directory 'path'.
    
    :param str path: Path (directory) where the data.yaml file to be modified is located
    """
    with open(os.path.join(path, "data.yaml"), "r") as f:
        lines = f.readlines()
    
    lines[0] = f"train: {os.path.join(os.pardir, path, 'train', 'images')}\n"
    lines[1] = f"val: {os.path.join(os.pardir, path, 'valid', 'images')}\n"

    with open(os.path.join(path, "data.yaml"), "w") as f:
        f.writelines(lines)

## YOLOv5 - Setup


Clone the YOLOv5 GitHub repository.

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Install the dependencies of YOLOv5.

In [ ]:
!pip3 install -r yolov5/requirements.txt

## Oxford Pets - By Species

* Source: https://public.roboflow.com/object-detection/oxford-pets/2/download/yolov5pytorch
* \# of classes: 2

In [ ]:
oxford_pets_species_zip_filename = "oxford_pets_species.zip"
oxford_pets_species_extract_dir = "oxford_pets_species"

Extract the zip file to the specified target directory.

In [ ]:
oxford_pets_species_zip_path = os.path.join(datasets_base_path, oxford_pets_species_zip_filename)
oxford_pets_species_extract_path = os.path.join(datasets_base_path, oxford_pets_species_extract_dir)

!unzip {oxford_pets_species_zip_path} -d {oxford_pets_species_extract_path}

Adapt the paths in the `data.yaml` to fit our project structure.

In [ ]:
adapt_data_yaml_train_val_keys(oxford_pets_extract_path)

Set general hyperparameters.

In [ ]:
epochs = 300
optimizer = "SGD"
batch_size = 32

Fine-tune pre-trained `YOLOv5n`.

In [ ]:
weights = "yolov5n.pt"
project_name = f"yolov5n_oxford_pets_species"
!python yolov5/train.py --weights {weights} --optimizer {optimizer} --batch {batch_size} --epochs {epochs} --data {oxford_pets_species_extract_path}/data.yaml --project {project} --name {project_name} --device {device} --cache

Fine-tune pre-trained `YOLOv5s`.

In [ ]:
weights = "yolov5s.pt"
project_name = f"yolov5s_oxford_pets_species"
!python yolov5/train.py --weights {weights} --optimizer {optimizer} --batch {batch_size} --epochs {epochs} --data {oxford_pets_species_extract_path}/data.yaml --project {project} --name {project_name} --device {device} --cache